In [108]:
import pandas as pd
import numpy as np
import warnings
from datetime import datetime
from datetime import date
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from xgboost import XGBClassifier
from datetime import datetime
from datetime import date
from sklearn.cluster import KMeans
import warnings
from scipy import stats
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import seaborn as sns
from scipy.stats import chi2_contingency
from collections import Counter


In [109]:
# Set options to display all rows
pd.set_option('display.max_rows', None)

# Set options to display all columns
pd.set_option('display.max_columns', None)

# Ignore warnings
warnings.filterwarnings("ignore")

In [110]:
# Load the datasets
raw_train_X = pd.read_csv("../data/raw/airbnb_train_x.csv")
raw_train_y =pd.read_csv("../data/raw/airbnb_train_y.csv")
test_x = pd.read_csv("../data/raw/airbnb_test_x.csv")

In [111]:
train = pd.concat([raw_train_X, raw_train_y], axis=1)
train['high_booking_rate'] = train['high_booking_rate'].map({'NO': 0, 'YES': 1})
train.drop(columns=['perfect_rating_score'], inplace = True)

In [112]:
# train.info()

In [113]:
from textblob import TextBlob

def compute_sentiment_score(text):
    """
    Returns the sentiment polarity score of a given text.
    Range: [-1.0, 1.0] (negative to positive sentiment)
    """
    try:
        return TextBlob(str(text)).sentiment.polarity
    except:
        return 0.0

def add_sentiment_features(df, columns):
    """
    Adds sentiment score features to the DataFrame for given text columns.

    Parameters:
    df (pd.DataFrame): Input dataframe
    columns (list): List of column names to compute sentiment on

    Returns:
    pd.DataFrame: DataFrame with new sentiment columns appended
    """
    df = df.copy()
    for col in columns:
        sentiment_col = f"sentiment_{col}"
        df[sentiment_col] = df[col].apply(compute_sentiment_score)
    return df

In [114]:
# adding sentiment features - 

# Columns to apply sentiment to
text_cols = ["description", "summary", "house_rules", "host_about", "neighborhood_overview"]

# Add sentiment features
train = add_sentiment_features(train, text_cols)

test_x = add_sentiment_features(test_x, text_cols)

In [115]:
# print(train.shape)
# print(test_x.shape)

In [116]:
def add_static_binary_amenities_features(df, ensure_columns=True):
    # Top 20 amenities (as seen in bar chart)
    top_amenities = [
        'Wireless Internet', 'Heating', 'Kitchen', 'Essentials', 'Internet', 'TV', 'Shampoo',
        'Smoke detector', 'Air conditioning', 'Hangers', 'Washer', 'Dryer', 'Iron',
        'Carbon monoxide detector', 'Hair dryer', 'Laptop friendly workspace',
        'Family/kid friendly', 'Fire extinguisher', 'Cable TV', 'First aid kit'
    ]

    # Top 8 features (as seen in bar chart)
    top_features = [
        'Host Has Profile Pic', 'Is Location Exact', 'Host Identity Verified', 'Instant Bookable',
        'Host Is Superhost', 'Requires License', 'Require Guest Phone Verification',
        'Require Guest Profile Picture'
    ]

    df['amenities'] = df['amenities'].fillna('').str.lower()
    df['features'] = df['features'].fillna('').str.lower()

    # Create amenity features
    for item in top_amenities:
        col_name = 'has_' + item.lower().replace(" ", "_")
        df[col_name] = df['amenities'].apply(lambda x: int(item.lower() in x))

    # Create feature features
    for item in top_features:
        col_name = 'has_' + item.lower().replace(" ", "_")
        df[col_name] = df['features'].apply(lambda x: int(item.lower() in x))

    # Ensure all expected columns exist (for test set alignment)
    if ensure_columns:
        expected_cols = ['has_' + x.lower().replace(" ", "_") for x in (top_amenities + top_features)]
        for col in expected_cols:
            if col not in df.columns:
                df[col] = 0

    return df


In [117]:
train = add_static_binary_amenities_features(train)
test_x = add_static_binary_amenities_features(test_x)

In [118]:
# print(train.shape)
# print(test_x.shape)

In [119]:
def encode_location_columns_train_test(train_df, test_df, city_thresh=400, state_thresh=50, target_col='high_booking_rate'):
    # ------------------ Separate Target ------------------
    y = train_df[target_col].copy()
    train_df = train_df.drop(columns=[target_col])

    # ------------------ Process State ------------------
    train_df['state'] = train_df['state'].str.upper()
    test_df['state'] = test_df['state'].str.upper()
    top_states = train_df['state'].value_counts()[train_df['state'].value_counts() > state_thresh].index.tolist()
    train_df['state_grouped'] = train_df['state'].apply(lambda x: x if x in top_states else 'OTHER')
    test_df['state_grouped'] = test_df['state'].apply(lambda x: x if x in top_states else 'OTHER')

    # ------------------ Process City ------------------
    train_df['city'] = train_df['city'].str.lower()
    test_df['city'] = test_df['city'].str.lower()
    top_cities = train_df['city'].value_counts()[train_df['city'].value_counts() > city_thresh].index.tolist()
    train_df['city_grouped'] = train_df['city'].apply(lambda x: x if x in top_cities else 'other')
    test_df['city_grouped'] = test_df['city'].apply(lambda x: x if x in top_cities else 'other')

    # ------------------ One-hot Encoding ------------------
    combined = pd.concat([train_df, test_df], axis=0, sort=False)
    combined = pd.get_dummies(combined, columns=['state_grouped', 'city_grouped'], drop_first=True)

    # Cast dummy columns to int
    dummy_cols = [col for col in combined.columns if col.startswith('state_grouped_') or col.startswith('city_grouped_')]
    combined[dummy_cols] = combined[dummy_cols].astype(int)

    # ------------------ Split Back ------------------
    train_processed = combined.iloc[:len(y)].copy()
    test_processed = combined.iloc[len(y):].copy()

    # Restore target column
    train_processed[target_col] = y.values

    return train_processed, test_processed


In [120]:
train, test_x= encode_location_columns_train_test(train, test_x)

In [121]:
print(train.shape)
print(test_x.shape)

(92067, 127)
(10000, 126)


In [122]:
def data_prep_1(df, drop_cols):

    # ------------------ dropping unncessary columns ------------------
    df = df.drop(columns = drop_cols,axis = 1)

    # ------------------ making interaction variables ------------------
    df['price_per_accommodate'] = df['price'] / (df['accommodates'] + 1e-5)  # small epsilon to avoid div by zero
    df['cleaning_fee_to_price'] = df['cleaning_fee'] / (df['price'] + 1e-5)
    df['bed_bath_ratio'] = df['beds'] / (df['bathrooms'] + 1e-5)
    df['availability_ratio'] = df['availability_30'] / (df['availability_365'] + 1e-5)
    df['has_security_deposit'] = (df['security_deposit'] > 0).astype(int)
    df['booking_pressure'] = (df['availability_365'] == 0).astype(int)
    df['availability_rate_30'] = df['availability_30'] / 30
    df['availability_rate_365'] = df['availability_365'] / 365
    df['high_cleaning_fee_flag'] = (df['cleaning_fee_to_price'] > 0.3).astype(int)
    df['long_term_availability_flag'] = (df['availability_365'] > 300).astype(int)
    df['bathrooms_per_guest'] = df['bathrooms'] / (df['guests_included'] + 1e-5)
    df['price_per_bedroom'] = df['price'] / (df['bedrooms'] + 1e-5)

    df['min_night_bin'] = pd.cut(df['minimum_nights'], bins=[0, 2, 7, 30, 100, 365], labels=False, include_lowest=True)
    # Host experience bins
    df['host_experience_bin'] = pd.cut(df['host_total_listings_count'], 
                                   bins=[-1, 5, 20, float('inf')], 
                                   labels=['new', 'experienced', 'superhost'])
    df['host_experience_bin'] = df['host_experience_bin'].astype('category')
    df['host_experience_encoded'] = df['host_experience_bin'].cat.codes



    # ------------------ numerical column transformations ------------------
    df['host_acceptance_rate'] = (df['host_acceptance_rate'].str.rstrip('%').astype(float).div(100))
    def bin_host_acceptance_rate(val):
        if pd.isna(val):
            return 0  
        elif val < 0.92:
            return 1  
        else:
            return 2  
    df['host_acceptance_rate_binned'] = df['host_acceptance_rate'].apply(bin_host_acceptance_rate).astype(int)

    df['security_deposit_binned'] = pd.cut(df['security_deposit'],bins=[0, 100, 150, 200, 249, 250, 300, 500, 999]).astype(str)
    df.loc[df['security_deposit'].isna(), 'security_deposit_binned'] = 'Missing'
    # Convert to categorical then get category codes
    df['security_deposit_binned'] = df['security_deposit_binned'].astype('category')
    df['security_deposit_binned_encoded'] = df['security_deposit_binned'].cat.codes
    
    # df['extra_people_log'] = df.log1p(df['extra_people'])

    # Bin guests_included
    df['guests_included_binned'] = pd.cut(df['guests_included'],bins=[-1, 1, 2, 4, 20],labels=["≤1", "1-2", "2-4", "4+"]).astype(str)
    df['guests_included_encoded'] = df['guests_included_binned'].astype('category').cat.codes

    # df['host_listings_count_log'] = np.log1p(df['host_listings_count'].astype(float))

    

    # ------------------ categorical column transformations ------------------

    # 1. host_response_time
    response_time_map = {
        'a few days or more': 0,
        'within a day': 1,
        'within a few hours': 2,
        'within an hour': 3
    }
    df['host_response_time_encoded'] = df['host_response_time'].map(response_time_map)


    # 2. room_type
    room_type_map = {
        'Entire home/apt': 0,
        'Private room': 1,
        'Shared room': 2
    }
    df['room_type_encoded'] = df['room_type'].map(room_type_map)
    

    # 3. property_type
    top_categories = df['property_type'].value_counts().nlargest(10).index
    df['property_type_grouped'] = df['property_type'].apply(lambda x: x if x in top_categories else 'Other')
    df = pd.get_dummies(df, columns=['property_type_grouped'], drop_first=True)
    dummy_cols = [col for col in df.columns if col.startswith('property_type_grouped')]
    df[dummy_cols] = df[dummy_cols].astype(int)


    # 4. cancellation_policy
    rare_policies = ['no_refunds', 'super_strict_30', 'super_strict_60']
    # Replace rare with 'other'
    df['cancellation_policy_clean'] = df['cancellation_policy'].apply(lambda x: 'other' if x in rare_policies else x)
    cancellation_policy_mapping = {
        'other': 0,
        'flexible': 1,
        'moderate': 2,
        'strict': 3
    }
    df['cancellation_policy_encoded'] = df['cancellation_policy_clean'].map(cancellation_policy_mapping)


    # 5. bed_type
    bed_type_mapping = {
        'Real Bed': 0,
        'Futon': 1,
        'Pull-out Sofa': 2,
        'Airbed': 3,
        'Couch': 4
    }
    df['bed_type_encoded'] = df['bed_type'].map(bed_type_mapping)


    # ------------------ handling text columns ------------------
    df['has_access'] = df['access'].notna().astype(int)
    df['description_length'] = df['description'].fillna('').apply(len)
    df['host_about_length'] = df['host_about'].fillna('').apply(len)
    df['has_interaction'] = df['interaction'].notna().astype(int)
    df['house_rules_length'] = df['house_rules'].fillna('').apply(len)
    df['has_space'] = df['space'].notna().astype(int)


    
    # ------------------ handling list of string columns ------------------
    df['features_count'] = df['features'].fillna('').apply(lambda x: len(str(x).split(',')))
    df['amenities_count'] = df['amenities'].fillna('').apply(lambda x: len(str(x).split(',')))

    

    # ------------------ handling date columns ------------------
    today = datetime.today()
    df['host_since'] = pd.to_datetime(df['host_since'], errors='coerce')
    df['first_review'] = pd.to_datetime(df['first_review'], errors='coerce')
    df['host_account_age'] = (today - df['host_since']).dt.days.fillna(0) 
    df['listing_age_days'] = (today - df['first_review']).dt.days.fillna(0)


    # ------------------ handling location columns ------------------

    # #1. state
    # df['state'] = df['state'].str.upper()
    # df = pd.get_dummies(df, columns=['state'], drop_first=True)
    # state_dummies = [col for col in df.columns if col.startswith('state_')]
    # df[state_dummies] = df[state_dummies].astype(int)


    # #2. city 
    # df['city'] = df['city'].str.lower()
    # # Group rare cities into 'other'
    # top_cities = df['city'].value_counts()[df['city'].value_counts() > 400].index
    # df['city_grouped'] = df['city'].apply(lambda x: x if x in top_cities else 'other')
    # # One-hot encode
    # df = pd.get_dummies(df, columns=['city_grouped'], drop_first=True)
    # #Convert one-hot columns to integers
    # city_dummies = [col for col in df.columns if col.startswith('city_grouped_')]
    # df[city_dummies] = df[city_dummies].astype(int)


    
    # ------------------ drop more columns ------------------
    df = df.drop(columns = ['description','host_about', ], axis = 1)
    df = df.drop(columns=['host_acceptance_rate'], axis = 1)
    df = df.drop(columns=['license'], axis = 1)
    # df = df.drop(columns=['monthly_price'], axis = 1)
    df = df.drop(columns=['security_deposit','security_deposit_binned'], axis = 1)
    df = df.drop(columns=['square_feet'], axis = 1)
    # df = df.drop(columns=['weekly_price'], axis = 1)
    # df = df.drop(columns=['extra_people'], axis = 1)
    df = df.drop(['guests_included_binned'], axis=1)
    # df = df.drop(columns=['host_listings_count'], axis=1)
    # df = df.drop(columns=['host_total_listings_count'], axis = 1)
    df = df.drop(columns=['host_since','first_review'], axis = 1)
    df = df.drop(columns=['experiences_offered'], axis = 1)
    df = df.drop(columns=['host_response_time'], axis = 1)
    df = df.drop(columns=['room_type'], axis = 1)
    df = df.drop(columns=['property_type'], axis = 1)
    df = df.drop(columns=['cancellation_policy', 'cancellation_policy_clean'], axis = 1)
    df = df.drop(columns=['bed_type'], axis = 1)
    df.drop(columns=['state'], inplace=True)
    df.drop(columns=['city'], inplace=True)
    df.drop(columns=['access'], inplace=True)
    df.drop(columns=['space'], inplace=True)
    df.drop(columns=['interaction'], inplace=True)
    df.drop(columns=['house_rules'], inplace=True)
    df.drop(columns=['host_experience_bin'], inplace=True)
    df.drop(columns=['features'], inplace=True)
    df.drop(columns=['amenities'], inplace=True)

    missing_summary = df.isnull().sum()
    print(missing_summary[missing_summary > 0])
    

    return df

In [123]:
text_cols_to_drop = ['host_name', 'name', 'neighborhood_overview', 'notes', 'street', 'summary','transit']
loc_cols_to_drop = ['latitude','longitude','country', 'host_location', 'country_code','zipcode','neighborhood_group','jurisdiction_names','host_neighbourhood', 'neighborhood','smart_location','market']


In [124]:
drop_list = text_cols_to_drop + loc_cols_to_drop 

In [125]:
train = data_prep_1(train,drop_list)
test_x = data_prep_1(test_x,drop_list)

host_response_rate            13450
host_total_listings_count       169
host_verifications              169
accommodates                     50
bathrooms                       281
bedrooms                        113
beds                             91
price                           533
weekly_price                  73556
monthly_price                 69637
cleaning_fee                  20180
price_per_accommodate           583
cleaning_fee_to_price         20620
bed_bath_ratio                  333
bathrooms_per_guest             281
price_per_bedroom               646
min_night_bin                     8
host_response_time_encoded    13450
dtype: int64
host_response_rate            1444
host_total_listings_count       15
host_verifications              16
accommodates                     5
bathrooms                       33
bedrooms                        12
beds                            10
price                           52
weekly_price                  8008
monthly_price           

In [132]:
def advanced_interaction_feature_creation(df):

    df['desc_sentiment_price'] = df['sentiment_description'] * df['price']
    df['summary_sentiment_price'] = df['sentiment_summary'] * df['price']
    df['desc_sentiment_per_accommodate'] = df['sentiment_description'] / (df['accommodates'] + 1e-5)
    df['summary_sentiment_per_accommodate'] = df['sentiment_summary'] / (df['accommodates'] + 1e-5)

    df['availability_price_ratio'] = df['availability_365'] / (df['price'] + 1e-5)
    df['superhost_high_cleaning'] = ((df['has_host_is_superhost'] == 1) & (df['high_cleaning_fee_flag'] == 1)).astype(int)

    df['is_luxury_property'] = ((df['price'] > 500)).astype(int)
    df['is_fast_response_premium'] = ((df['host_response_time_encoded'] == 3) & (df['price'] > 200)).astype(int)

    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    return df

In [133]:
train_new = advanced_interaction_feature_creation(train)
test_x = advanced_interaction_feature_creation(test_x)

In [134]:
print(train_new.shape)
print(test_x.shape)

(92067, 134)
(10000, 133)


In [135]:
import numpy as np
import pandas as pd
from collections import Counter

# def process_host_verifications(df, col_name='host_verifications', top_n=10, prefix='verif_'):
#     """
#     Processes a comma-separated string column of host verifications into one-hot encoded top verifications
#     and a grouped 'other' category, while preserving missing values.

#     Parameters:
#     - df: pandas DataFrame (will be modified in-place)
#     - col_name: str, name of the input column (e.g., 'host_verifications')
#     - top_n: int, number of top verifications to one-hot encode
#     - prefix: str, prefix for new binary columns (e.g., 'verif_')

#     Returns:
#     - list of new column names created
#     """
    
#     # Step 1: Convert string to list, preserve NaNs
#     col_list_name = f'{col_name}_list'
#     df[col_list_name] = df[col_name].dropna().apply(lambda x: [v.strip().lower() for v in x.split(',')])
#     df[col_list_name] = df[col_list_name].fillna(np.nan)

#     # Step 2: Find top N most common verifications
#     all_verifs = df[col_list_name].dropna().explode()
#     top_verifs = [v for v, c in Counter(all_verifs).most_common(top_n)]

#     # Step 3: Create binary columns for top verifications
#     new_cols = []
#     for v in top_verifs:
#         new_col = f'{prefix}{v}'
#         df[new_col] = df[col_list_name].apply(lambda x: int(v in x) if isinstance(x, list) else np.nan)
#         new_cols.append(new_col)

#     # Step 4: Create 'other' column for rare verifications
#     other_col = f'{prefix}other'
#     df[other_col] = df[col_list_name].apply(
#         lambda x: int(any(v not in top_verifs for v in x)) if isinstance(x, list) else np.nan
#     )
#     new_cols.append(other_col)

#     # Step 5: Drop intermediate column
#     df.drop(columns=[col_name, col_list_name], inplace=True)

#     return new_cols


def process_host_verifications(df, top_n=10, prefix='verif_', known_cols=None):
    col_name = 'host_verifications'

    col_list_name = f'{col_name}_list'
    df[col_list_name] = df[col_name].dropna().apply(lambda x: [v.strip().lower() for v in x.split(',')])
    df[col_list_name] = df[col_list_name].fillna(np.nan)

    if known_cols:
        top_verifs = [col[len(prefix):] for col in known_cols if col != f'{prefix}other']
    else:
        all_verifs = df[col_list_name].dropna().explode()
        top_verifs = [v for v, c in Counter(all_verifs).most_common(top_n)]

    new_cols = []
    for v in top_verifs:
        new_col = f'{prefix}{v}'
        df[new_col] = df[col_list_name].apply(lambda x: int(v in x) if isinstance(x, list) else 0)
        new_cols.append(new_col)

    other_col = f'{prefix}other'
    df[other_col] = df[col_list_name].apply(
        lambda x: int(any(v not in top_verifs for v in x)) if isinstance(x, list) else 0
    )
    new_cols.append(other_col)

    df.drop(columns=[col_name, col_list_name], inplace=True)

    return df, new_cols


In [136]:
train_new, verif_cols = process_host_verifications(train_new, top_n=10)
test_x, _ = process_host_verifications(test_x, top_n=10, known_cols=verif_cols)


In [137]:
train_new.shape

(92067, 144)

In [138]:
test_x.shape

(10000, 143)

In [139]:
train_new.head()

,host_response_rate,host_listings_count,host_total_listings_count,accommodates,bathrooms,bedrooms,beds,price,weekly_price,monthly_price,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,availability_30,availability_60,availability_90,availability_365,sentiment_description,sentiment_summary,sentiment_house_rules,sentiment_host_about,sentiment_neighborhood_overview,has_wireless_internet,has_heating,has_kitchen,has_essentials,has_internet,has_tv,has_shampoo,has_smoke_detector,has_air_conditioning,has_hangers,has_washer,has_dryer,has_iron,has_carbon_monoxide_detector,has_hair_dryer,has_laptop_friendly_workspace,has_family/kid_friendly,has_fire_extinguisher,has_cable_tv,has_first_aid_kit,has_host_has_profile_pic,has_is_location_exact,has_host_identity_verified,has_instant_bookable,has_host_is_superhost,has_requires_license,has_require_guest_phone_verification,has_require_guest_profile_picture,state_grouped_CO,state_grouped_DC,state_grouped_IL,state_grouped_LA,state_grouped_MA,state_grouped_NY,state_grouped_OR,state_grouped_OTHER,state_grouped_TN,state_grouped_TX,state_grouped_WA,city_grouped_boston,city_grouped_bronx,city_grouped_brooklyn,city_grouped_chicago,city_grouped_denver,city_grouped_long beach,city_grouped_los angeles,city_grouped_nashville,city_grouped_new orleans,city_grouped_new york,city_grouped_oakland,city_grouped_other,city_grouped_pasadena,city_grouped_portland,city_grouped_queens,city_grouped_san diego,city_grouped_san francisco,city_grouped_santa monica,city_grouped_seattle,city_grouped_washington,city_grouped_west hollywood,high_booking_rate,price_per_accommodate,cleaning_fee_to_price,bed_bath_ratio,availability_ratio,has_security_deposit,booking_pressure,availability_rate_30,availability_rate_365,high_cleaning_fee_flag,long_term_availability_flag,bathrooms_per_guest,price_per_bedroom,min_night_bin,host_experience_encoded,host_acceptance_rate_binned,security_deposit_binned_encoded,guests_included_encoded,host_response_time_encoded,room_type_encoded,property_type_grouped_Bed & Breakfast,property_type_grouped_Bungalow,property_type_grouped_Condominium,property_type_grouped_Dorm,property_type_grouped_Guesthouse,property_type_grouped_House,property_type_grouped_Loft,property_type_grouped_Other,property_type_grouped_Townhouse,cancellation_policy_encoded,bed_type_encoded,has_access,description_length,host_about_length,has_interaction,house_rules_length,has_space,features_count,amenities_count,host_account_age,listing_age_days,desc_sentiment_price,summary_sentiment_price,desc_sentiment_per_accommodate,summary_sentiment_per_accommodate,availability_price_ratio,superhost_high_cleaning,is_luxury_property,is_fast_response_premium,verif_reviews,verif_phone,verif_email,verif_kba,verif_jumio,verif_facebook,verif_government_id,verif_work_email,verif_google,verif_linkedin,verif_other
0,100.0,2,2.0,2.0,1.0,2.0,1.0,70.0,NaN,NaN,10.0,1,0,3,1125,12,42,72,72,0.070833,0.187500,0.258333,0.203704,-0.033333,1,1,1,1,1,1,0,1,1,1,0,1,0,1,1,1,0,0,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,34.999825,0.142857,0.99999,0.166667,1,0,0.400000,0.197260,0,0,0.999990,34.999825,1.0,0,0,0,3,3.0,1,0,0,0,0,0,0,0,0,0,3,0,1,926,552,1,104,1,4,14,3594.0,3086,4.958333,13.125000,0.035416,0.093750,1.028571,0,0,0,1,1,1,1,1,0,0,0,0,0,0
1,100.0,2,2.0,4.0,1.0,2.0,2.0,250.0,NaN,NaN,125.0,2,20,5,29,0,0,0,0,0.240616,0.172222,0.009495,0.251947,0.330195,1,1,1,1,1,1,1,1,0,1,0,1,1,1,1,1,0,1,1,1,1,1,1,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,62.499844,0.500000,1.99998,0.000000,1,1,0.000000,0.000000,1,0,0.499998,124.999375,1.0,0,0,6,0,2.0,0,0,0,1,0,0,0,0,0,0,3,0,1,999,2129,1,677,1,6,20,5824.0,4963,60.154113,43.055556,0.060154,0.043055,0.000000,1,0,0,1,1,1,1,0,1,0,1,0,0,0
2,NaN,2,2.0,2.0,1.0,1.0,1.0,100.0,NaN,NaN,20.0,1,0,3,1125,30,60,90,365,0.302443,0.175000,0.153114,0.000000,0.449536,1,1,1,1,1,1,1,0,1,0,1,1,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0

In [140]:
test_x.head()

,host_response_rate,host_listings_count,host_total_listings_count,accommodates,bathrooms,bedrooms,beds,price,weekly_price,monthly_price,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,availability_30,availability_60,availability_90,availability_365,sentiment_description,sentiment_summary,sentiment_house_rules,sentiment_host_about,sentiment_neighborhood_overview,has_wireless_internet,has_heating,has_kitchen,has_essentials,has_internet,has_tv,has_shampoo,has_smoke_detector,has_air_conditioning,has_hangers,has_washer,has_dryer,has_iron,has_carbon_monoxide_detector,has_hair_dryer,has_laptop_friendly_workspace,has_family/kid_friendly,has_fire_extinguisher,has_cable_tv,has_first_aid_kit,has_host_has_profile_pic,has_is_location_exact,has_host_identity_verified,has_instant_bookable,has_host_is_superhost,has_requires_license,has_require_guest_phone_verification,has_require_guest_profile_picture,state_grouped_CO,state_grouped_DC,state_grouped_IL,state_grouped_LA,state_grouped_MA,state_grouped_NY,state_grouped_OR,state_grouped_OTHER,state_grouped_TN,state_grouped_TX,state_grouped_WA,city_grouped_boston,city_grouped_bronx,city_grouped_brooklyn,city_grouped_chicago,city_grouped_denver,city_grouped_long beach,city_grouped_los angeles,city_grouped_nashville,city_grouped_new orleans,city_grouped_new york,city_grouped_oakland,city_grouped_other,city_grouped_pasadena,city_grouped_portland,city_grouped_queens,city_grouped_san diego,city_grouped_san francisco,city_grouped_santa monica,city_grouped_seattle,city_grouped_washington,city_grouped_west hollywood,price_per_accommodate,cleaning_fee_to_price,bed_bath_ratio,availability_ratio,has_security_deposit,booking_pressure,availability_rate_30,availability_rate_365,high_cleaning_fee_flag,long_term_availability_flag,bathrooms_per_guest,price_per_bedroom,min_night_bin,host_experience_encoded,host_acceptance_rate_binned,security_deposit_binned_encoded,guests_included_encoded,host_response_time_encoded,room_type_encoded,property_type_grouped_Bed & Breakfast,property_type_grouped_Bungalow,property_type_grouped_Condominium,property_type_grouped_Dorm,property_type_grouped_Guesthouse,property_type_grouped_House,property_type_grouped_Loft,property_type_grouped_Other,property_type_grouped_Townhouse,cancellation_policy_encoded,bed_type_encoded,has_access,description_length,host_about_length,has_interaction,house_rules_length,has_space,features_count,amenities_count,host_account_age,listing_age_days,desc_sentiment_price,summary_sentiment_price,desc_sentiment_per_accommodate,summary_sentiment_per_accommodate,availability_price_ratio,superhost_high_cleaning,is_luxury_property,is_fast_response_premium,verif_reviews,verif_phone,verif_email,verif_kba,verif_jumio,verif_facebook,verif_government_id,verif_work_email,verif_google,verif_linkedin,verif_other
0,80.0,2,2.0,6.0,1.5,2.0,3.0,600.0,NaN,10000.0,100.0,1,0,2,60,30,60,90,365,0.252079,0.000000,0.0,0.283006,0.000000,1,1,1,1,1,1,1,1,0,0,1,1,0,1,0,0,0,1,1,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,99.999833,0.166667,1.999987,0.082192,0,0,1.0,1.000000,0,1,1.499985,299.99850,0,0,0,8,3,2.0,0,0,0,0,0,0,1,0,0,0,3,0,0,999,721,0,61,1,3,18,4460.0,3564,151.247540,0.000000,0.042013,0.000000,0.608333,0,1,0,1,1,1,1,0,1,0,1,0,0,0
1,100.0,1,1.0,3.0,1.0,1.0,1.0,105.0,NaN,NaN,NaN,2,15,2,1125,0,3,11,11,0.016667,0.233333,0.0,0.000000,0.000000,1,1,1,1,1,1,0,1,1,1,1,1,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,34.999883,NaN,0.999990,0.000000,0,0,0.0,0.030137,0,0,0.499998,104.99895,0,0,2,8,0,3.0,0,0,0,1,0,0,0,0,0,0,1,0,1,780,0,0,8,1,3,15,3190.0,3171,1.750000,24.500000,0.005556,0.077778,0.104762,0,0,0,1,1,1,0,1,0,0,0,0,1,0
2,NaN,2,2.0,2.0,1.5,1.0,1.0,55.0,NaN,NaN,15.0,2,15,2,1125,0,0,0,0,0.397778,0.614286,0.0,0.200606,0.233333,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,1,1,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,27.499863,0.272727,0.666662,0.00

In [141]:
import re

def clean_column_names(df):
    """
    Cleans a dataframe's column names:
    - Replaces spaces and special characters with underscores
    - Removes any multiple consecutive underscores
    - Converts to lowercase (optional: remove if you want original casing)
    """
    df.columns = [
        re.sub(r'\W+', '_', col).strip('_')  # Replace non-alphanumeric with underscores, remove trailing _
        for col in df.columns
    ]
    return df

In [142]:
train_final1 = clean_column_names(train_new)
test_final1 = clean_column_names(test_x)

In [143]:
#output this data into a csv as train_data_proccessed_1
train_final1.to_csv('../data/processed/train_data_new_v8.csv', index=False)
test_final1.to_csv('../data/processed/test_data_x_submission8.csv', index=False)

### Filtering out some features based on XGB importance

In [44]:
# drop_features = ['guests_included','security_deposit_binned_encoded','verif_jumio','verif_work_email','beds',
#                  'verif_government_id','verif_linkedin','has_space','property_type_grouped_Condominium',
#                  'verif_other','verif_kba','has_access','verif_facebook','long_term_availability_flag',
#                  'property_type_grouped_Bungalow','city_grouped_pasadena','property_type_grouped_Dorm',
#                  'state_MD']

# train_final2 = train_final1.drop(columns = drop_features, axis = 1)

In [45]:
train_final1.shape

(92067, 136)

In [46]:
# #output this data into a csv as train_data_proccessed_1
# train_final1.to_csv('../data/processed/train_data_new_v4.csv', index=False)

### Additional features - text columns 